In [1]:
import pandas as pd
import datetime
import yfinance as yf
import backtrader as bt
import numpy as np
import warnings

In [2]:
from Data_Preprocess import IAM_data_setup, download_IAM, SelectIndex, generate_interval
from Data_Preprocess import calculate_returns, calculate_date_range

In [3]:
assets, asset_classes = IAM_data_setup()

In [4]:
display(asset_classes['Sector'].unique())
display(asset_classes['Class'].unique())

array(['Consumer Defensive', 'Real Estate', 'Industrials', 'Technology',
       'Cash', 'Municipal', 'Financial Services', 'Energy', 'Index',
       'Corporate', 'Healthcare', 'Consumer Cyclical', 'International',
       'Treasury', 'Ultrashort', 'Basic Materials', 'High Yield', 'ETF',
       'Agency'], dtype=object)

array(['Equity', 'Cash', 'Fixed Income', 'Commodity', 'Crypto'],
      dtype=object)

In [5]:
print(assets)

['0341.HK', '0405.HK', '0737.HK', '0P00001DU6', '0P00002C5C', '0P00006OXK.T', '0P0000UIC0', '0P0000VP5F', '0P0000YVJG', '0P0000Z5AZ', '0P0000ZJFK', '0P00013N6C', '0P0001DFIZ', '0P0001DHN3', '0P0001Q45W.HK', '0P0001Q87U', '1186.HK', '1810.HK', '3175.HK', '3187.HK', '9988.HK', 'ADDYY', 'AGGU.L', 'ARKG', 'ARKK', 'BABA', 'BIL', 'CSBGU7.SW', 'ERND.L', 'GLD', 'GOVT', 'HYGH', 'HYLD.L', 'HYMB', 'IAGG', 'IBIT', 'IBTS.SW', 'IDEV', 'IDTL.L', 'IDTM.L', 'IEF', 'IEMB.L', 'IGIL.L', 'IGSB', 'IHYU.L', 'ISAG.L', 'ISTB', 'IVE', 'IWRD.L', 'JDN.MX', 'LQDE.L', 'METV', 'PAVE', 'SDHY.L', 'SGOV', 'SHV', 'SHY', 'TFLO', 'TLT', 'USO', 'VGIT', 'ZM']


In [6]:
display(asset_classes.shape)
display(asset_classes.T)

(62, 6)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
Assets,0341.HK,0405.HK,0737.HK,0P00001DU6,0P00002C5C,0P00006OXK.T,0P0000UIC0,0P0000VP5F,0P0000YVJG,0P0000Z5AZ,...,TFLO,USO,ZM,TLT,BIL,IEF,VGIT,SGOV,SHY,0P0001Q45W.HK
Class,Equity,Equity,Equity,Equity,Cash,Equity,Fixed Income,Fixed Income,Fixed Income,Equity,...,Fixed Income,Commodity,Equity,Fixed Income,Fixed Income,Fixed Income,Fixed Income,Fixed Income,Fixed Income,Cash
Sector,Consumer Defensive,Real Estate,Industrials,Technology,Cash,Industrials,Municipal,Financial Services,Energy,Index,...,Treasury,Energy,Technology,Treasury,Treasury,Treasury,Treasury,Treasury,Treasury,Cash
Type,Stock,RIT,Stock,Fund,Fund,Fund,Fund,Fund,Fund,Fund,...,ETF,ETF,Stock,ETF,ETF,ETF,ETF,ETF,ETF,Fund
Currency,HKD,HKD,HKD,USD,USD,JPY,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,HKD
Rate,0.128205,0.128205,0.128205,1.0,1.0,0.0066,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.128205


In [7]:
asset_classes[asset_classes['Assets'] == '0P0001Q45W.HK']

,Assets,Class,Sector,Type,Currency,Rate
61,0P0001Q45W.HK,Cash,Cash,Fund,HKD,0.128205


In [9]:
# Download and generate full data file from yfinance
# Date range
start = '2013-01-01'
end = '2024-09-30'

df = yf.download(assets, start=start, end=end)

[*********************100%***********************]  62 of 62 completed


In [10]:
df.to_csv("IAM_full_data.txt")

In [15]:
# Downloading data
data = download_IAM()
start = data.index[0]
end = data.index[-1]
print(start, " , ", end)
display(data.shape)
display(data.head())
display(data.tail())

2013-01-02 00:00:00+00:00  ,  2024-09-27 00:00:00+00:00


(3030, 372)

Price                      Adj Close                                 \
Ticker                       0341.HK   0405.HK   0737.HK 0P00001DU6   
Date                                                                  
2013-01-02 00:00:00+00:00  15.133362  1.645048  2.076490        NaN   
2013-01-03 00:00:00+00:00  15.237492  1.649469  2.086059        NaN   
2013-01-04 00:00:00+00:00  15.202783  1.649469  2.081275        NaN   
2013-01-07 00:00:00+00:00  15.202783  1.689269  2.105197        NaN   
2013-01-08 00:00:00+00:00  15.306908  1.680425  2.095629        NaN   

Price                                                                    \
Ticker                    0P00002C5C 0P00006OXK.T 0P0000UIC0 0P0000VP5F   
Date                                                                      
2013-01-02 00:00:00+00:00        NaN          NaN        NaN        NaN   
2013-01-03 00:00:00+00:00        NaN          NaN        NaN        NaN   
2013-01-04 00:00:00+00:00        NaN          NaN        NaN        NaN   
2013-01-07 00:00:00+00:00        NaN          NaN        NaN        NaN   
2013-01-08 00:00:00+00:00        NaN          NaN        NaN        NaN   

Price                                            ... Volume              \
Ticker                    0P0000YVJG 0P0000Z5AZ  ...   PAVE SDHY.L SGOV   
Date                                             ...                      
2013-01-02 00:00:00+00:00        NaN        NaN  ...    NaN    NaN  NaN   
2013-01-03 00:00:00+00:00        NaN        NaN  ...    NaN    NaN  NaN   
2013-01-04 00:00:00+00:00        NaN        NaN  ...    NaN    NaN  NaN   
2013-01-07 00:00:00+00:00        NaN        NaN  ...    NaN    NaN  NaN   
2013-01-08 00:00:00+00:00        NaN        NaN  ...    NaN    NaN  NaN   

Price                                                                      \
Ticker                          SHV        SHY TFLO         TLT       USO   
Date                                                                        
2013-01-02 00:00:00+00:00  259200.0  2469100.0  NaN   7472700.0  693100.0   
2013-01-03 00:00:00+00:00  357500.0  1816100.0  NaN  10004300.0  489425.0   
2013-01-04 00:00:00+00:00  303100.0  1028700.0  NaN   9574600.0  742613.0   
2013-01-07 00:00:00+00:00  288400.0   672300.0  NaN   4193100.0  356675.0   
2013-01-08 00:00:00+00:00  140900.0   712300.0  NaN   4380900.0  480188.0   

Price                                    
Ticker                         VGIT  ZM  
Date                                     
2013-01-02 00:00:00+00:00   50900.0 NaN  
2013-01-03 00:00:00+00:00   23300.0 NaN  
2013-01-04 00:00:00+00:00   35000.0 NaN  
2013-01-07 00:00:00+00:00   26000.0 NaN  
2013-01-08 00:00:00+00:00  130100.0 NaN  

[5 rows x 372 columns]

Price                     Adj Close                                        \
Ticker                      0341.HK 0405.HK 0737.HK 0P00001DU6 0P00002C5C   
Date                                                                        
2024-09-23 00:00:00+00:00      7.73    0.95    1.86  83.570000     1.9333   
2024-09-24 00:00:00+00:00      7.76    0.98    1.86  85.919998     1.9336   
2024-09-25 00:00:00+00:00      7.87    1.01    1.83  86.209999     1.9339   
2024-09-26 00:00:00+00:00      8.04    1.03    1.85  89.349998     1.9341   
2024-09-27 00:00:00+00:00      8.36    1.11    1.85  91.949997     1.9344   

Price                                                                    \
Ticker                    0P00006OXK.T 0P0000UIC0 0P0000VP5F 0P0000YVJG   
Date                                                                      
2024-09-23 00:00:00+00:00          NaN  19.179701  18.190001      15.33   
2024-09-24 00:00:00+00:00       2876.0        NaN  18.200001      15.33   
2024-09-25 00:00:00+00:00       2862.0        NaN  18.190001      15.33   
2024-09-26 00:00:00+00:00       2945.0        NaN  18.200001      15.33   
2024-09-27 00:00:00+00:00       2860.0        NaN  18.219999      15.33   

Price                                  ...     Volume                      \
Ticker                     0P0000Z5AZ  ...       PAVE   SDHY.L       SGOV   
Date                                   ...                                  
2024-09-23 00:00:00+00:00  289.839996  ...   628600.0   1364.0  5220900.0   
2024-09-24 00:00:00+00:00  290.820007  ...   586100.0   1697.0  3923500.0   
2024-09-25 00:00:00+00:00  291.339996  ...  1364500.0  10793.0  3372400.0   
2024-09-26 00:00:00+00:00  294.070007  ...   699300.0  32704.0  4364400.0   
2024-09-27 00:00:00+00:00  292.700012  ...   939400.0   3897.0  4945400.0   

Price                                                                   \
Ticker                           SHV        SHY       TFLO         TLT   
Date                                                                     
2024-09-23 00:00:00+00:00  3188200.0  5030800.0  1797400.0  35703900.0   
2024-09-24 00:00:00+00:00  3068800.0  8947900.0  1876900.0  31005400.0   
2024-09-25 00:00:00+00:00  3090200.0  4829100.0   975800.0  36738200.0   
2024-09-26 00:00:00+00:00  3071100.0  2321800.0  1301200.0  33332300.0   
2024-09-27 00:00:00+00:00  4247700.0  3208500.0  2162400.0  28317600.0   

Price                                                       
Ticker                           USO       VGIT         ZM  
Date                                                        
2024-09-23 00:00:00+00:00  2837700.0  2391600.0  2526600.0  
2024-09-24 00:00:00+00:00  2387000.0  1570400.0  2294700.0  
2024-09-25 00:00:00+00:00  3469200.0  1787400.0  2479900.0  
2024-09-26 00:00:00+00:00  4593800.0  1907800.0  2018900.0  
2024-09-27 00:00:00+00:00  3214700.0  1617600.0  1417500.0  

[5 rows x 372 columns]

In [12]:
#  all test is from the 1004th day and finish at the last testdata day.
#
start_test = 1004
end_test = data.shape[0] - 1
test_size = 1000
print(f" Testing data from {start_test} to {end_test}")

 Testing data from 1004 to 3029


In [13]:
rebalance_index = {}
rebalance_index["M"] = SelectIndex(data, mode="M", start_index=start_test)
rebalance_index["Q"] = SelectIndex(data, mode="Q", start_index=start_test)
rebalance_index["S"] = SelectIndex(data, mode="S", start_index=start_test)
print(rebalance_index)

{'M': [1009, 1030, 1051, 1071, 1094, 1113, 1136, 1158, 1179, 1202, 1223, 1245, 1267, 1287, 1309, 1329, 1350, 1371, 1394, 1415, 1437, 1460, 1480, 1503, 1525, 1545, 1567, 1587, 1608, 1629, 1652, 1672, 1695, 1717, 1738, 1761, 1782, 1803, 1825, 1845, 1867, 1888, 1909, 1931, 1954, 1975, 1997, 2019, 2040, 2062, 2082, 2102, 2125, 2146, 2167, 2189, 2211, 2233, 2255, 2276, 2298, 2321, 2342, 2362, 2385, 2405, 2427, 2449, 2470, 2493, 2515, 2536, 2558, 2579, 2600, 2620, 2643, 2662, 2685, 2707, 2728, 2751, 2772, 2794, 2816, 2836, 2858, 2879, 2899, 2921, 2944, 2964, 2987, 3009, 3029], 'Q': [1030, 1094, 1158, 1223, 1287, 1350, 1415, 1480, 1545, 1608, 1672, 1738, 1803, 1867, 1931, 1997, 2062, 2125, 2189, 2255, 2321, 2385, 2449, 2515, 2579, 2643, 2707, 2772, 2836, 2899, 2964, 3029], 'S': [1030, 1158, 1287, 1415, 1545, 1672, 1803, 1931, 2062, 2189, 2321, 2449, 2579, 2707, 2836, 2964]}


In [14]:
r_index={}
r_index["M"] = generate_interval(data, rebalance_index["M"], "rebalance_index_M.csv")
r_index["Q"] = generate_interval(data, rebalance_index["Q"], "rebalance_index_Q.csv")
r_index["S"] = generate_interval(data, rebalance_index["S"], "rebalance_index_S.csv")
r_index["S"]

,Date
index,
1030,2016-12-30 00:00:00+00:00
1158,2017-06-30 00:00:00+00:00
1287,2017-12-29 00:00:00+00:00
1415,2018-06-29 00:00:00+00:00
1545,2018-12-31 00:00:00+00:00
1672,2019-06-28 00:00:00+00:00
1803,2019-12-31 00:00:00+00:00
1931,2020-06-30 00:00:00+00:00
2062,2020-12-31 00:00:00+00:00


In [17]:
returns = calculate_returns(data)
display(returns)
returns.to_csv("IAM_Returns.txt")

/home/thomas/projects/Fund_Portfolio_Allocation/Data_Preprocess.py:78: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return c_data.pct_change().round(4)


,0341.HK,0405.HK,0737.HK,0P00001DU6,0P00002C5C,0P00006OXK.T,0P0000UIC0,0P0000VP5F,0P0000YVJG,0P0000Z5AZ,...,PAVE,SDHY.L,SGOV,SHV,SHY,TFLO,TLT,USO,VGIT,ZM
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03 00:00:00+00:00,0.0069,0.0027,0.0046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0000,NaN,-0.0136,-0.0024,-0.0032,NaN
2013-01-04 00:00:00+00:00,-0.0023,0.0000,-0.0023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0001,NaN,0.0039,0.0041,-0.0002,NaN
2013-01-07 00:00:00+00:00,0.0000,0.0241,0.0115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.0001,0.0000,NaN,0.0004,0.0012,0.0008,NaN
2013-01-08 00:00:00+00:00,0.0068,-0.0052,-0.0045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0001,NaN,0.0066,0.0012,0.0018,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-23 00:00:00+00:00,-0.0026,-0.0206,0.0137,0.0059,0.0004,0.0000,0.0064,0.0006,0.0007,0.0010,...,0.0025,0.0021,0.0003,0.0002,0.0000,0.0002,-0.0021,-0.0092,-0.0005,0.0187
2024-09-24 00:00:00+00:00,0.0039,0.0316,0.0000,0.0281,0.0002,-0.0066,0.0000,0.0005,0.0000,0.0034,...,0.0027,-0.0023,0.0001,0.0001,0.0010,0.0002,-0.0002,0.0144,0.0015,0.0093
2024-09-25 00:00:00+00:00,0.0142,0.0306,-0.0161,0.0034,0.0002,-0.0049,0.0000,-0.0005,0.0000,0.0018,...,-0.0057,0.0012,0.0000,0.0002,-0.0008,0.0002,-0.0083,-0.0247,-0.0028,-0.0213


In [18]:
df = calculate_date_range(data, returns.columns.to_list())
display(df)


Process 0341.HK


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2013-01-02 00:00:00+00:00,15.133362,21.799999,22.15,21.700001,22.000000,1298000.0
2013-01-03 00:00:00+00:00,15.237492,21.950001,22.00,21.850000,21.950001,246000.0
2013-01-04 00:00:00+00:00,15.202783,21.900000,22.00,21.900000,21.950001,396000.0
2013-01-07 00:00:00+00:00,15.202783,21.900000,22.10,21.850000,21.950001,148000.0
2013-01-08 00:00:00+00:00,15.306908,22.049999,22.15,21.850000,21.850000,260000.0
...,...,...,...,...,...,...
2024-09-23 00:00:00+00:00,7.730000,7.730000,7.78,7.690000,7.750000,493203.0
2024-09-24 00:00:00+00:00,7.760000,7.760000,7.79,7.690000,7.780000,1246000.0
2024-09-25 00:00:00+00:00,7.870000,7.870000,7.95,7.800000,7.800000,2154001.0


Process 0405.HK


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2013-01-02 00:00:00+00:00,1.645048,3.677888,3.687774,3.638340,3.638340,8265569.0
2013-01-03 00:00:00+00:00,1.649469,3.687774,3.687774,3.668001,3.677888,7148539.0
2013-01-04 00:00:00+00:00,1.649469,3.687774,3.697661,3.668001,3.687774,7226193.0
2013-01-07 00:00:00+00:00,1.689269,3.776756,3.806416,3.697661,3.697661,17962684.0
2013-01-08 00:00:00+00:00,1.680425,3.756982,3.796529,3.737208,3.776756,9059890.0
...,...,...,...,...,...,...
2024-09-23 00:00:00+00:00,0.950000,0.950000,0.970000,0.950000,0.970000,1925637.0
2024-09-24 00:00:00+00:00,0.980000,0.980000,0.980000,0.960000,0.960000,2636255.0
2024-09-25 00:00:00+00:00,1.010000,1.010000,1.020000,0.990000,1.000000,9546278.0


Process 0737.HK


...

Process VGIT


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2013-01-02 00:00:00+00:00,51.734016,65.349998,65.370003,65.279999,65.349998,50900.0
2013-01-03 00:00:00+00:00,51.567806,65.139999,65.349998,65.139999,65.349998,23300.0
2013-01-04 00:00:00+00:00,51.559868,65.129997,65.160004,65.059998,65.099998,35000.0
2013-01-07 00:00:00+00:00,51.599453,65.180000,65.199997,65.110001,65.110001,26000.0
2013-01-08 00:00:00+00:00,51.694416,65.300003,65.300003,65.190002,65.190002,130100.0
...,...,...,...,...,...,...
2024-09-23 00:00:00+00:00,60.348495,60.540001,60.590000,60.389999,60.500000,2391600.0
2024-09-24 00:00:00+00:00,60.438213,60.630001,60.639999,60.419998,60.450001,1570400.0
2024-09-25 00:00:00+00:00,60.268745,60.459999,60.560001,60.459999,60.560001,1787400.0


Process ZM


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2019-04-18 00:00:00+00:00,62.000000,62.000000,66.000000,60.320999,65.000000,25764700.0
2019-04-22 00:00:00+00:00,65.699997,65.699997,68.900002,59.939999,61.000000,9949700.0
2019-04-23 00:00:00+00:00,69.000000,69.000000,74.168999,65.550003,66.870003,6786500.0
2019-04-24 00:00:00+00:00,63.200001,63.200001,71.500000,63.160000,71.400002,4973500.0
2019-04-25 00:00:00+00:00,65.000000,65.000000,66.849998,62.599998,64.739998,3863300.0
...,...,...,...,...,...,...
2024-09-23 00:00:00+00:00,68.790001,68.790001,68.959999,67.510002,67.660004,2526600.0
2024-09-24 00:00:00+00:00,69.430000,69.430000,70.169998,68.651001,69.410004,2294700.0
2024-09-25 00:00:00+00:00,67.949997,67.949997,69.360001,67.785004,68.959999,2479900.0


,start,i_start,end,i_end
Symbol,,,,
0341.HK,2013-01-02 00:00:00+00:00,0,2024-09-27 00:00:00+00:00,3029
0405.HK,2013-01-02 00:00:00+00:00,0,2024-09-27 00:00:00+00:00,3029
0737.HK,2013-01-02 00:00:00+00:00,0,2024-09-27 00:00:00+00:00,3029
0P00001DU6,2019-01-02 00:00:00+00:00,1546,2024-09-27 00:00:00+00:00,3029
0P00002C5C,2019-01-02 00:00:00+00:00,1546,2024-09-27 00:00:00+00:00,3029
...,...,...,...,...
TFLO,2014-02-04 00:00:00+00:00,281,2024-09-27 00:00:00+00:00,3029
TLT,2013-01-02 00:00:00+00:00,0,2024-09-27 00:00:00+00:00,3029
USO,2013-01-02 00:00:00+00:00,0,2024-09-27 00:00:00+00:00,3029


In [19]:
print(returns.shape)
print(data.shape)

(3030, 62)
(3030, 372)
